# Name: Margaret Nguyen

# Machine Learning: Ordinary least squares (OLS)

**Assignment: Create an Ordinary Least Squares model with cyclist deaths and injuries per capita as the target variable. Use per capita parameters in the model and employ data from both Massachusetts and Pennsylvania.**

In [57]:
# Import packages
import numpy as np # v 1.21.5
import sklearn # v 1.0.2
import pandas as pd # v 1.4.4
import ydata_profiling as pp # v 3.6.6
import statsmodels.api as sm # v 0.13.2

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# Ploting libraries 
import matplotlib.pyplot as plt # v 3.5.2
import seaborn as sns # v 0.11.2

%matplotlib inline

## I. df_pa_crash.csv (Pennsylvannia)

### 1. Load and clean data

In [58]:
# Read the csv file 
df_pa_crash = pd.read_csv('./data/df_pa_crash.csv')

# Clean datasets
df_pa_crash = df_pa_crash.drop(columns = ['Unnamed: 0'])

# Select columns with numeric data types (int or float) using select_dtypes
numeric_columns = df_pa_crash.select_dtypes(include=['number'])

# Create a new DataFrame with only the numeric columns
df_pa_filtered = df_pa_crash[numeric_columns.columns]

# Drop unnessary columns
df_pa_filtered = df_pa_filtered.drop(['PENN_DOT_MUNI_ID', 'state', 'county', 'county_subdivision', 'LAND_AREA.1', 'PENN_DOT_COUNTY_NUM', 'FEDERAL_EIN_CODE', 'HOME_RULE_YEAR', 'INCORPORATION_YEAR', 'MUNICIPALITY'], axis=1)

# Replace NaN values with 0 in the entire DataFrame
df_pa_filtered = df_pa_filtered.fillna(0)

# Rename BNA Score column to BNA_SCORE column
df_pa_filtered.rename(columns={'BNA Score': 'BNA_SCORE'}, inplace=True)

# Reset index
df_pa_filtered.reset_index(inplace = True, drop = True)

In [59]:
# Define the columns for which you want to calculate per capita values
columns_to_convert = [
    'LAND_AREA', 'BIKE_TO_WORK_EST', 'BIKE_TO_WORK_MARG',
    'WALK_TO_WORK_EST', 'WALK_TO_WORK_MARG', 'DRIVE_SOLO_TO_WORK_EST',
    'DRIVE_SOLO_TO_WORK_MARG', 'CARPOOL_TO_WORK_EST',
    'CARPOOL_TO_WORK_MARG', 'PUBTRANS_TO_WORK_EST',
    'PUBTRANS_TO_WORK_MARG', 'EMPLOYEES_FULL_TIME',
    'EMPLOYEES_PART_TIME', 'AUTOMOBILE_COUNT',
    'BICYCLE_BY_AUTO_COUNT', 'BICYCLE_DEATH_BY_AUTO_COUNT',
    'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'PED_BY_AUTO_COUNT',
    'PED_DEATH_BY_AUTO_COUNT', 'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT',
    'BICYCLE_SOLO_COUNT', 'BICYCLE_DEATH_SOLO_COUNT',
    'BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT', 'PED_SOLO_COUNT',
    'PED_DEATH_SOLO_COUNT', 'PED_SUSP_SERIOUS_INJ_SOLO_COUNT'
]

# Create new columns with "_PER_CAPITA" suffix by dividing each column by 'POPULATION'
for column in columns_to_convert:
    new_column_name = column + '_PER_CAPITA'
    df_pa_filtered[new_column_name] = df_pa_filtered[column] / df_pa_filtered['POPULATION']

**Perform Exploratory Data Analysis (EDA) to check for multicollinearity among the independent variables in the dataset**

In [60]:
# df_pa_filtered.columns
df_pa_filtered.shape, df_pa_filtered.dtypes

((44, 54),
 POPULATION                                             int64
 LAND_AREA                                            float64
 BIKE_TO_WORK_EST                                       int64
 BIKE_TO_WORK_MARG                                      int64
 WALK_TO_WORK_EST                                       int64
 WALK_TO_WORK_MARG                                      int64
 DRIVE_SOLO_TO_WORK_EST                                 int64
 DRIVE_SOLO_TO_WORK_MARG                                int64
 CARPOOL_TO_WORK_EST                                    int64
 CARPOOL_TO_WORK_MARG                                   int64
 PUBTRANS_TO_WORK_EST                                   int64
 PUBTRANS_TO_WORK_MARG                                  int64
 EMPLOYEES_FULL_TIME                                  float64
 EMPLOYEES_PART_TIME                                  float64
 AUTOMOBILE_COUNT                                     float64
 BICYCLE_DEATH_BY_AUTO_COUNT                          float

In [61]:
df_pa_filtered.describe()

,POPULATION,LAND_AREA,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,...,BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT_PER_CAPITA,PED_BY_AUTO_COUNT_PER_CAPITA,PED_DEATH_BY_AUTO_COUNT_PER_CAPITA,PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT_PER_CAPITA,BICYCLE_SOLO_COUNT_PER_CAPITA,BICYCLE_DEATH_SOLO_COUNT_PER_CAPITA,BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT_PER_CAPITA,PED_SOLO_COUNT_PER_CAPITA,PED_DEATH_SOLO_COUNT_PER_CAPITA,PED_SUSP_SERIOUS_INJ_SOLO_COUNT_PER_CAPITA
count,4.400000e+01,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,...,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000
mean,6.692270e+04,11.077273,421.522727,69.954545,2236.681818,222.613636,17178.113636,657.727273,2626.772727,318.250000,...,0.000025,0.000112,0.000025,0.000087,0.000020,0.000003,0.000016,0.000104,0.000030,0.000074
std,2.413978e+05,21.859731,2143.309650,150.068659,8366.840010,345.704889,52187.648923,812.147580,8449.989660,476.840675,...,0.000042,0.000122,0.000050,0.000103,0.000041,0.000015,0.000034,0.000106,0.000050,0.000082
min,2.590000e+02,0.200000,0.000000,6.000000,0.000000,7.000000,107.000000,62.000000,11.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.969750e+03,1.675000,0.000000,11.000000,27.500000,27.000000,1967.250000,237.000000,231.000000,129.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.440800e+04,4.150000,13.000000,21.500000,222.000000,107.500000,5540.500000,489.000000,461.000000,189.500000,...,0.000000,0.000091,0.000000,0.000072,0.000000,0.000000,0.000000,0.000089,0.000000,0.000054
75%,4.129250e+04,9.975000,73.750000,53.250000,1165.500000,268.750000,13169.000000,811.500000,1593.500000,343.250000,...,0.000033,0.000187,0.000024,0.000153,0.000022,0.000000,0.000017,0.000178,0.000052,0.000137
max,1.596865e+06,134.100000,14172.000000,940.000000,54269.000000,2031.000000,343702.000000,5207.000000,55482.000000,3121.000000,...,0.000201,0.000500,0.000215,0.000400,0.000182,0.000091,0.000171,0.000430,0.000225,0.000344


In [62]:
# Check for NaN missing values
df_pa_filtered.isna().sum()

POPULATION                                           0
LAND_AREA                                            0
BIKE_TO_WORK_EST                                     0
BIKE_TO_WORK_MARG                                    0
WALK_TO_WORK_EST                                     0
WALK_TO_WORK_MARG                                    0
DRIVE_SOLO_TO_WORK_EST                               0
DRIVE_SOLO_TO_WORK_MARG                              0
CARPOOL_TO_WORK_EST                                  0
CARPOOL_TO_WORK_MARG                                 0
PUBTRANS_TO_WORK_EST                                 0
PUBTRANS_TO_WORK_MARG                                0
EMPLOYEES_FULL_TIME                                  0
EMPLOYEES_PART_TIME                                  0
AUTOMOBILE_COUNT                                     0
BICYCLE_DEATH_BY_AUTO_COUNT                          0
BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT               0
PED_DEATH_BY_AUTO_COUNT                              0
PED_SUSP_S

In [63]:
df_pa_filtered.head(3)

,POPULATION,LAND_AREA,BIKE_TO_WORK_EST,BIKE_TO_WORK_MARG,WALK_TO_WORK_EST,WALK_TO_WORK_MARG,DRIVE_SOLO_TO_WORK_EST,DRIVE_SOLO_TO_WORK_MARG,CARPOOL_TO_WORK_EST,CARPOOL_TO_WORK_MARG,...,BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT_PER_CAPITA,PED_BY_AUTO_COUNT_PER_CAPITA,PED_DEATH_BY_AUTO_COUNT_PER_CAPITA,PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT_PER_CAPITA,BICYCLE_SOLO_COUNT_PER_CAPITA,BICYCLE_DEATH_SOLO_COUNT_PER_CAPITA,BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT_PER_CAPITA,PED_SOLO_COUNT_PER_CAPITA,PED_DEATH_SOLO_COUNT_PER_CAPITA,PED_SUSP_SERIOUS_INJ_SOLO_COUNT_PER_CAPITA
0,125250,17.6,79,51,2396,607,36549,1464,8407,817,...,0.000032,0.000248,0.000016,0.000232,0.000000,0.0,0.000000,0.000168,0.000008,0.000160
1,44114,9.9,4,6,761,210,14943,816,1391,269,...,0.000023,0.000136,0.000023,0.000113,0.000045,0.0,0.000045,0.000136,0.000000,0.000136
2,3268,1.8,0,11,0,11,1397,231,109,79,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


**Use Pandas Profiling**

In [64]:
#pp.ProfileReport(df_pa_filtered)

**According to the Pandas Profiling report, we can see that our independent variables are highly correlated with each other. This suggests a high likelihood of multicollinearity in our linear regression model. To address this issue, I need to remove some independent variables and retain only the necessary ones as much as possible. Additionally, I have decided to remove BICYCLE_DEATH_BY_AUTO_COUNT and BICYCLE_DEATH_SOLO_COUNT because both exhibit a high level of imbalance (62.6%).**

In [65]:
# Drop independent variables that are highly correlated
df_pa_filtered = df_pa_filtered.drop(['LAND_AREA', 'BIKE_TO_WORK_EST', 'BIKE_TO_WORK_MARG',
        'WALK_TO_WORK_EST', 'WALK_TO_WORK_MARG', 'DRIVE_SOLO_TO_WORK_EST',
        'DRIVE_SOLO_TO_WORK_MARG', 'CARPOOL_TO_WORK_EST',
        'CARPOOL_TO_WORK_MARG', 'PUBTRANS_TO_WORK_EST', 'PUBTRANS_TO_WORK_MARG',
        'EMPLOYEES_FULL_TIME', 'EMPLOYEES_PART_TIME', 'AUTOMOBILE_COUNT',
        'BICYCLE_BY_AUTO_COUNT', 'BICYCLE_DEATH_BY_AUTO_COUNT',
        'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'PED_BY_AUTO_COUNT',
        'PED_DEATH_BY_AUTO_COUNT', 'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT',
        'BICYCLE_SOLO_COUNT', 'BICYCLE_DEATH_SOLO_COUNT',
        'BICYCLE_SUSP_SERIOUS_INJ_SOLO_COUNT', 'PED_SOLO_COUNT',
        'PED_DEATH_SOLO_COUNT', 'PED_SUSP_SERIOUS_INJ_SOLO_COUNT'], axis=1)

### 2. Fit the linear regression using [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

**Separate data set in Y(independent) and X (dependent) variable**

In [66]:
y = df_pa_filtered["BICYCLE_BY_AUTO_COUNT_PER_CAPITA"] # Y = df_pa_filtered.BICYCLE_BY_AUTO_COUNT_PER_CAPITA
X = df_pa_filtered.loc[:, df_pa_filtered.columns != "BICYCLE_BY_AUTO_COUNT_PER_CAPITA"] # I want all columns except the BICYCLE_BY_AUTO_COUNT_PER_CAPITA column which is the dependent variable

**Use the train_test_split function to split your data into training (80%) and testing set (20%)**

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

**Fit, run or estimate the regression model**

In [68]:
model = LinearRegression() # Create an instance of the linear regression class
model.fit(X_train, y_train)

LinearRegression()

In [69]:
# Coefficient values
model.coef_

array([ 5.50944437e-24, -9.84252285e-19,  9.69570474e-17,  2.21150211e-17,
       -2.82775332e-18, -5.55493471e-18,  2.73386964e-17,  1.13401830e-18,
       -4.69830647e-18,  2.42943939e-18, -9.35111139e-18, -1.03867415e-19,
        1.09073425e-17, -4.88870359e-19,  1.18219098e-17, -2.09029757e-17,
        1.00000000e+00,  1.00000000e+00,  1.03067376e-13, -1.03865976e-13,
       -1.02511150e-13, -7.15408215e-14,  6.93299803e-14,  7.41850325e-14,
       -2.50541269e-13,  2.49987338e-13,  2.50531607e-13])

In [70]:
print(model.intercept_, model.coef_,model.score(X_test, y_test)) # Model score gives you the R-squared

2.7064396730669404e-17 [ 5.50944437e-24 -9.84252285e-19  9.69570474e-17  2.21150211e-17
 -2.82775332e-18 -5.55493471e-18  2.73386964e-17  1.13401830e-18
 -4.69830647e-18  2.42943939e-18 -9.35111139e-18 -1.03867415e-19
  1.09073425e-17 -4.88870359e-19  1.18219098e-17 -2.09029757e-17
  1.00000000e+00  1.00000000e+00  1.03067376e-13 -1.03865976e-13
 -1.02511150e-13 -7.15408215e-14  6.93299803e-14  7.41850325e-14
 -2.50541269e-13  2.49987338e-13  2.50531607e-13] 1.0


**Using the independent variables in the testing set, to predict the dependent variables**

In [71]:
y_pred = model.predict(X_test)

**Use the test set, check how well my model does in terms of error metrics**

In [72]:
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
MAPE = mean_absolute_percentage_error(y_test,y_pred)
MSE, MAE, MAPE

(3.842137959126562e-34, 1.489982740580677e-17, 0.0570693556336695)

### 3. Fit the linear regression using [Statsmodels](https://www.statsmodels.org/stable/index.html)

In [73]:
X_train = sm.add_constant(X_train)

In [74]:
model2 = sm.OLS(y_train, X_train).fit()

In [75]:
print(model2.summary())

                                   OLS Regression Results                                   
Dep. Variable:     BICYCLE_BY_AUTO_COUNT_PER_CAPITA   R-squared:                       1.000
Model:                                          OLS   Adj. R-squared:                  1.000
Method:                               Least Squares   F-statistic:                 1.151e+27
Date:                              Sun, 26 Nov 2023   Prob (F-statistic):          2.71e-134
Time:                                      23:19:04   Log-Likelihood:                 1407.1
No. Observations:                                35   AIC:                            -2764.
Df Residuals:                                    10   BIC:                            -2725.
Df Model:                                        24                                         
Covariance Type:                          nonrobust                                         
                                                        coef    std er

**In-sample prediction**

In [76]:
ypred2 = model2.predict(X_train)
model2.params

const                                                2.710505e-19
POPULATION                                          -8.310580e-24
BNA_SCORE                                            5.442187e-20
LAND_AREA_PER_CAPITA                                -6.158268e-17
BIKE_TO_WORK_EST_PER_CAPITA                         -1.301043e-17
BIKE_TO_WORK_MARG_PER_CAPITA                        -1.289116e-16
WALK_TO_WORK_EST_PER_CAPITA                          2.298509e-17
WALK_TO_WORK_MARG_PER_CAPITA                        -5.724587e-17
DRIVE_SOLO_TO_WORK_EST_PER_CAPITA                   -6.776264e-18
DRIVE_SOLO_TO_WORK_MARG_PER_CAPITA                   2.355429e-17
CARPOOL_TO_WORK_EST_PER_CAPITA                       1.675092e-17
CARPOOL_TO_WORK_MARG_PER_CAPITA                     -1.376937e-17
PUBTRANS_TO_WORK_EST_PER_CAPITA                     -1.107919e-17
PUBTRANS_TO_WORK_MARG_PER_CAPITA                     4.987330e-17
EMPLOYEES_FULL_TIME_PER_CAPITA                      -3.989864e-17
EMPLOYEES_

**According to the results from Statsmodels:**
- **The estimated coefficients, which include DRIVE_SOLO_TO_WORK_EST_PER_CAPITA, CARPOOL_TO_WORK_MARG_PER_CAPITA, AUTOMOBILE_COUNT_PER_CAPITA, PED_BY_AUTO_COUNT_PER_CAPITA, BICYCLE_DEATH_SOLO_COUNT_PER_CAPITA, and PED_SUSP_SERIOUS_INJ_SOLO_COUNT_PER_CAPITA, are statistically significant at the 5% level of significance in the 90% training set.**
- **Additionally, in the 80% training set, the estimated coefficients, including DRIVE_SOLO_TO_WORK_EST_PER_CAPITA, AUTOMOBILE_COUNT_PER_CAPITA, and PED_BY_AUTO_COUNT_PER_CAPITA, are statistically significant at the 5% level of significance.**
- **However, in the 70% training set, none of the estimated coefficients are statistically insignificant.**

### 4. Plot the residuals, display feature coefficients, and create a QQ plot

### Credit:

The following code is based on the work of my supervisor, Mitch Shiles. The original code can be found at this link: [Mitch Shiles' GitHub](https://github.com/rmshiles/Custom-Data-Tools/blob/main/testing%20for%20normality%20.ipynb).

**Below is a defined function designed to test residuals for a normal distribution.**

In [77]:
# Define a test for normality 

# Test for normality 
# y_test is the target variable and y_pred are the predicted variables 
def test_for_normality(y_test,y_pred):
    from scipy.stats import boxcox
    from scipy.stats import jarque_bera
    from scipy.stats import normaltest
    colo = np.random.randint(3, size=1)
    colors=[['r','gold','c','m'],
            ['g','orange','b','hotpink'],
            ['skyblue','coral','lightgreen','mediumslateblue'],
           ['g','limegreen','orange','yellow']]
    
    try:
        data_series = y_pred-y_test
    except:
        data_series=y_test
    # Input the mean, standard deviation and lenght of the residuals
    normal = np.random.normal(np.mean(data_series), np.std(data_series), len(data_series))

    plt.figure(figsize=(16, 12))

    plt.subplot(2, 2, 1)
    plt.hist(data_series,color = colors[colo[0]][1],alpha = 0.8) #bins=40,
    plt.hist(normal,color = colors[colo[0]][2], alpha = 0.2)

    # Generate a Box Plot of solar system counts
    plt.subplot(2, 2, 2)
    plt.boxplot(data_series)

    # Generate a QQ plot of the gamma distribution and the solar system counts 
    plt.subplot(2, 1, 2)
    orderd_normal = sorted(normal)
    ordered_data=sorted(data_series)
    plt.scatter(ordered_data,orderd_normal, color = colors[colo[0]][3])
    plt.plot(orderd_normal,orderd_normal,color= colors[colo[0]][0])
    plt.title('QQPlot of residuals and normal Distribution')
    plt.xlabel('residuals')
    plt.ylabel('normaly distribution')
    plt.show()

    jb_stats = jarque_bera(data_series)
    norm_stats = normaltest(data_series)
    print('the Jarque berra stat is {}, and the pvalue is {}'.format(jb_stats[0],jb_stats[1]))
    print(norm_stats)
    
    # elecResiduals = np.sort(result2_elect.resid[np.logical_not(np.isnan(result2_elect.resid))])
    sorted_data_series = np.sort(data_series[np.logical_not(np.isnan(data_series))])

    data_series_min = sorted_data_series.min()
    data_series_max = sorted_data_series.max()
    data_series_len = len(sorted_data_series)
    data_series_std = np.std(sorted_data_series)
    data_series_avg = np.mean(sorted_data_series)

    print('the Minimum is {}'.format(data_series_min))
    print('the Maximum is {}'.format(data_series_max))
    print('the Length is {}'.format(data_series_len))
    print('the Length is {}'.format(data_series_len))
    print('the Standard Deviation is {}'.format(data_series_std))
    print('the Mean is {}'.format(data_series_avg))
    print('\n')

In [78]:
import matplotlib
matplotlib.use('Qt5Agg')  # Use an appropriate backend like 'Qt5Agg' for GUI display
import matplotlib.pyplot as plt

In [79]:
# Plot the residuals, display feature coefficients, and create a QQ plot
test_for_normality(y_test,y_pred)

the Jarque berra stat is 1.0312352396086275, and the pvalue is 0.5971316803540168
NormaltestResult(statistic=3.6507931534719367, pvalue=0.16115372163861147)
the Minimum is -3.190588185512868e-17
the Maximum is 6.987754203356243e-18
the Length is 9
the Length is 9
the Standard Deviation is 1.478008399826186e-17
the Mean is -1.2874894675956763e-17




/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


**The warning regarding the kurtosis test indicates that it's typically valid for larger sample sizes (n >= 20). With a sample size of 9, it's recommended to use other statistical tests or methods to assess kurtosis accurately.**

## II. df_mass_bna.csv (Massachusetts)

### 1. Load and clean data

In [80]:
# Create a blank dataframe
df_mass_bna = pd.DataFrame()

# Read the csv file 
df_mass_bna = pd.read_csv('./data/df_mass_bna.csv')

# Clean datasets
df_mass_crash = df_mass_bna.drop(columns = ['Unnamed: 0'])

# Exclude the NaN from 'VEHC_CONFIG_CL'
df_mass_crash = df_mass_crash[df_mass_crash['VEHC_CONFIG_CL'].notna()]

# List of NOT automobiles: Snowmobile, Moped, Motorcycle, Other Light Trucks (10,000 lbs., or Less), Other e.g. Farm Equipment, Unknown.
# Exclude the non-automobiles from 'VEHC_CONFIG_CL' columns
list_non_automobiles = ['V1:(Unknown vehicle configuration)', 'V1:(Other e.g. farm equipment)', 'V1:(Unknown vehicle configuration) / V2:(Unknown vehicle configuration)']
df_mass_crash= df_mass_crash[~df_mass_crash['VEHC_CONFIG_CL'].isin(list_non_automobiles)]

/var/folders/vw/dbzntfg96b58pmmcc2cxykdh0000gn/T/ipykernel_9521/3164059786.py:5: DtypeWarning: Columns (26,34,35,68,97) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mass_bna = pd.read_csv('/Users/margaret06/Documents/GitHub/Carlisle_Borough_Transportation_Study/data/df_mass_bna.csv')


In [81]:
# Fatal - injuries that resulted in death 
# Incapacitating - serious injuries require immediate medical attention

## BICYCLE_DEATH_BY_AUTO_COUNT
# Filter the DataFrame for cyclist fatalities
cyclist_fatalities = df_mass_crash[(df_mass_crash['INJY_STAT_DESCR'] == 'Fatal injury (K)') & (df_mass_crash['NON_MTRST_TYPE_CL'] == 'Cyclist')]

# Group the filtered DataFrame by 'CITY_TOWN_NAME' and calculate the count for each city
bicycle_death_counts = cyclist_fatalities.groupby('CITY_TOWN_NAME').size().reset_index(name='BICYCLE_DEATH_BY_AUTO_COUNT')

# Merge the bicycle_death_counts DataFrame into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(bicycle_death_counts, on='CITY_TOWN_NAME', how='left')

## BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT
cyclist_incapacitating = df_mass_crash[(df_mass_crash['INJY_STAT_DESCR'] == 'Non-fatal injury - Incapacitating') & (df_mass_crash['NON_MTRST_TYPE_CL'] == 'Cyclist')]
bicycle_sus_serious_inj_counts = cyclist_incapacitating.groupby('CITY_TOWN_NAME').size().reset_index(name='BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT')

# Merge the bicycle_sus_serious_inj_counts DataFrame into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(bicycle_sus_serious_inj_counts, on='CITY_TOWN_NAME', how='left')

# Replace NaN values with 0 in the specified columns
df_mass_crash['BICYCLE_DEATH_BY_AUTO_COUNT'].fillna(0, inplace=True)
df_mass_crash['BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT'].fillna(0, inplace=True)

## BICYCLE_BY_AUTO_COUNT
df_mass_crash['BICYCLE_BY_AUTO_COUNT'] = df_mass_crash['BICYCLE_DEATH_BY_AUTO_COUNT'] + df_mass_crash['BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT']

## AUTOMOBILE_COUNT
auto_count = df_mass_crash.groupby('CITY_TOWN_NAME')['NUMB_VEHC'].sum().reset_index()
auto_count.rename(columns={'NUMB_VEHC': 'AUTOMOBILE_COUNT'}, inplace=True)
# Merge the auto_count into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(auto_count, on='CITY_TOWN_NAME', how='left')

## PED_DEATH_BY_AUTO_COUNT
# Filter the DataFrame for pedestrian fatalities
ped_fatalities = df_mass_crash[(df_mass_crash['INJY_STAT_DESCR'] == 'Fatal injury (K)') & (df_mass_crash['NON_MTRST_TYPE_CL'] == 'Pedestrian')]

# Group the filtered DataFrame by 'CITY_TOWN_NAME' and calculate the count for each city
ped_death_counts = cyclist_fatalities.groupby('CITY_TOWN_NAME').size().reset_index(name='PED_DEATH_BY_AUTO_COUNT')

# Merge the ped_death_counts DataFrame into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(ped_death_counts, on='CITY_TOWN_NAME', how='left')

## PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT
ped_incapacitating = df_mass_crash[(df_mass_crash['INJY_STAT_DESCR'] == 'Non-fatal injury - Incapacitating') & (df_mass_crash['NON_MTRST_TYPE_CL'] == 'Pedestrian')]
ped_sus_serious_inj_counts = ped_incapacitating.groupby('CITY_TOWN_NAME').size().reset_index(name='PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT')

# Merge the ped_sus_serious_inj_counts DataFrame into df_mass_crash on 'CITY_TOWN_NAME'
df_mass_crash = df_mass_crash.merge(ped_sus_serious_inj_counts, on='CITY_TOWN_NAME', how='left')

# Replace NaN values with 0 in the specified columns
df_mass_crash['PED_DEATH_BY_AUTO_COUNT'].fillna(0, inplace=True)
df_mass_crash['PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT'].fillna(0, inplace=True)

##PED_BY_AUTO_COUNT
df_mass_crash['PED_BY_AUTO_COUNT'] = df_mass_crash['PED_DEATH_BY_AUTO_COUNT'] + df_mass_crash['PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT']

# Drop the duplicated rows
df_mass_crash = df_mass_crash.drop_duplicates(subset=['CITY_TOWN_NAME', 'BNA Score', 'POPULATION', 
                                                      'BIKE_TO_WORK_EST', 'BICYCLE_BY_AUTO_COUNT', 
                                                      'BICYCLE_DEATH_BY_AUTO_COUNT', "AUTOMOBILE_COUNT",
                                                      'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 
                                                      'AUTOMOBILE_COUNT', 'PED_BY_AUTO_COUNT', 
                                                      'PED_DEATH_BY_AUTO_COUNT', 'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT'])

In [82]:
# Fix error
df_mass_crash = df_mass_crash.copy()

# Select columns with numeric data types (int or float) using select_dtypes
numeric_columns = df_mass_crash.select_dtypes(include=['number'])

# Create a new DataFrame with only the numeric columns
df_mass_crash = df_mass_crash[numeric_columns.columns]

# Rename BNA Score column to BNA_SCORE column
df_mass_crash.rename(columns={'BNA Score': 'BNA_SCORE'}, inplace=True)

# Reset index
df_mass_crash.reset_index(drop = True, inplace = True)

In [83]:
# Define the columns for which you want to calculate per capita values
columns_to_convert = [
    'BIKE_TO_WORK_EST', 'BIKE_TO_WORK_MARG',
    'WALK_TO_WORK_EST', 'WALK_TO_WORK_MARG', 'DRIVE_SOLO_TO_WORK_EST',
    'DRIVE_SOLO_TO_WORK_MARG', 'CARPOOL_TO_WORK_EST',
    'CARPOOL_TO_WORK_MARG', 'PUBTRANS_TO_WORK_EST',
    'PUBTRANS_TO_WORK_MARG', 'AUTOMOBILE_COUNT',
    'BICYCLE_BY_AUTO_COUNT', 'BICYCLE_DEATH_BY_AUTO_COUNT',
    'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'PED_BY_AUTO_COUNT',
    'PED_DEATH_BY_AUTO_COUNT', 'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT']

# Create new columns with "_PER_CAPITA" suffix by dividing each column by 'POPULATION'
for column in columns_to_convert:
    new_column_name = column + '_PER_CAPITA'
    df_mass_crash[new_column_name] = df_mass_crash[column] / df_mass_crash['POPULATION']

In [84]:
# Drop unnessary columns
df_mass_crash = df_mass_crash.drop(['OBJECTID', 'CRASH_NUMB', 'NUMB_VEHC', 'NUMB_NONFATAL_INJR',
       'NUMB_FATAL_INJR', 'MILEMARKER', 'X', 'Y', 'LAT', 'LON', 'YEAR',
       'DISTRICT_NUM', 'SPEED_LIMIT', 'AADT', 'AADT_YEAR', 'PK_PCT_SUT',
       'AV_PCT_SUT', 'PK_PCT_CT', 'AV_PCT_CT', 'LT_SIDEWLK', 'RT_SIDEWLK',
       'SHLDR_LT_W', 'SURFACE_WD', 'SHLDR_RT_W', 'NUM_LANES', 'OPP_LANES',
       'MED_WIDTH', 'PEAK_LANE', 'SPEED_LIM', 'STATN_NUM', 'OP_DIR_SL',
       'SHLDR_UL_W', 'VEHC_UNIT_NUMB', 'DRIVER_AGE', 'TOTAL_OCCPT_IN_VEHC',
       'PERS_NUMB', 'AGE', 'state', 'county', 'county_subdivision'], axis=1)

**Perform Exploratory Data Analysis (EDA) to check for multicollinearity among the independent variables in the dataset**

In [85]:
# Check for NaN missing values
df_mass_crash.isna().sum()

POPULATION                                           0
BIKE_TO_WORK_EST                                     0
BIKE_TO_WORK_MARG                                    0
WALK_TO_WORK_EST                                     0
WALK_TO_WORK_MARG                                    0
DRIVE_SOLO_TO_WORK_EST                               0
DRIVE_SOLO_TO_WORK_MARG                              0
CARPOOL_TO_WORK_EST                                  0
CARPOOL_TO_WORK_MARG                                 0
PUBTRANS_TO_WORK_EST                                 0
PUBTRANS_TO_WORK_MARG                                0
BNA_SCORE                                            0
BICYCLE_DEATH_BY_AUTO_COUNT                          0
BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT               0
BICYCLE_BY_AUTO_COUNT                                0
AUTOMOBILE_COUNT                                     0
PED_DEATH_BY_AUTO_COUNT                              0
PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT                   0
PED_BY_AUT

**Use Pandas Profiling**

In [86]:
#pp.ProfileReport(df_mass_crash)

In [87]:
# Drop independent variables that are highly correlated
df_mass_filtered = df_mass_crash.drop(['BIKE_TO_WORK_EST', 'BIKE_TO_WORK_MARG',
       'WALK_TO_WORK_EST', 'WALK_TO_WORK_MARG', 'DRIVE_SOLO_TO_WORK_EST',
       'DRIVE_SOLO_TO_WORK_MARG', 'CARPOOL_TO_WORK_EST',
       'CARPOOL_TO_WORK_MARG', 'PUBTRANS_TO_WORK_EST', 'PUBTRANS_TO_WORK_MARG',
       'BNA_SCORE', 'BICYCLE_DEATH_BY_AUTO_COUNT',
       'BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'BICYCLE_BY_AUTO_COUNT',
       'AUTOMOBILE_COUNT', 'PED_DEATH_BY_AUTO_COUNT',
       'PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT', 'PED_BY_AUTO_COUNT'], axis=1)

### 2. Fit the linear regression using [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

**Separate data set in Y(independent) and X (dependent) variable**

In [88]:
y2 = df_mass_filtered["BICYCLE_BY_AUTO_COUNT_PER_CAPITA"] # Y = df_mass_filtered.BICYCLE_BY_AUTO_COUNT_PER_CAPITA
X2 = df_mass_filtered.loc[:, df_mass_filtered.columns != "BICYCLE_BY_AUTO_COUNT_PER_CAPITA"] # I want all columns except the BICYCLE_BY_AUTO_COUNT_PER_CAPITA column which is the dependent variable

**Use the train_test_split function to split your data into training (80%) and testing set (20%)**

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, random_state=5)

**Fit, run or estimate the regression model**

In [90]:
model = LinearRegression() # Create an instance of the linear regression class
model.fit(X_train, y_train)

LinearRegression()

In [91]:
model.coef_

array([ 3.44574766e-23, -3.98579824e-17,  1.28142003e-16,  1.28850652e-17,
       -3.87454046e-17, -9.21571847e-19, -1.44165008e-17, -1.40552412e-17,
        1.63045372e-17,  7.69783542e-18,  7.17691016e-18,  8.60662766e-17,
        4.00000000e-01,  1.00000000e+00,  2.00000000e-01,  4.00000000e-01,
       -2.00000000e-01])

In [92]:
print(model.intercept_, model.coef_,model.score(X_test, y_test)) # Model score gives you the R-squared

-2.337810934421869e-18 [ 3.44574766e-23 -3.98579824e-17  1.28142003e-16  1.28850652e-17
 -3.87454046e-17 -9.21571847e-19 -1.44165008e-17 -1.40552412e-17
  1.63045372e-17  7.69783542e-18  7.17691016e-18  8.60662766e-17
  4.00000000e-01  1.00000000e+00  2.00000000e-01  4.00000000e-01
 -2.00000000e-01] 1.0


**Use the independent variables in the testing set, to predict the dependent variables**

In [93]:
y_pred = model.predict(X_test)

**Use the test set, check how well the model does in terms of error metrics**

In [94]:
MAE = mean_absolute_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred)
MAPE =  mean_absolute_percentage_error(y_test,y_pred)
MSE, MAE, MAPE

(2.4736008873257636e-36, 1.3799107858491724e-18, 0.002503289116764399)

### 3. Fit the linear regression using [Statsmodels](https://www.statsmodels.org/stable/index.html)

In [95]:
X_train = sm.add_constant(X_train)
model2 = sm.OLS(y_train, X_train).fit()

In [96]:
print(model2.summary())

                                   OLS Regression Results                                   
Dep. Variable:     BICYCLE_BY_AUTO_COUNT_PER_CAPITA   R-squared:                       1.000
Model:                                          OLS   Adj. R-squared:                  1.000
Method:                               Least Squares   F-statistic:                 4.470e+20
Date:                              Sun, 26 Nov 2023   Prob (F-statistic):          7.03e-202
Time:                                      23:19:07   Log-Likelihood:                 1145.8
No. Observations:                                36   AIC:                            -2260.
Df Residuals:                                    20   BIC:                            -2234.
Df Model:                                        15                                         
Covariance Type:                          nonrobust                                         
                                                        coef    std er

**According to the results from Statsmodels, in the 80% training set, the estimated coefficients, including BICYCLE_DEATH_BY_AUTO_COUNT_PER_CAPITA, BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT_PER_CAPITA, PED_BY_AUTO_COUNT_PER_CAPITA, PED_DEATH_BY_AUTO_COUNT_PER_CAPITA, PED_SUSP_SERIOUS_INJ_BY_AUTO_COUNT_PER_CAPITA,  are statistically significant at the 5% level of significance.**

**In-sample prediction**

In [97]:
ypred2 = model2.predict(X_train)
model2.params

const                                               -2.066760e-19
POPULATION                                          -2.841655e-20
BIKE_TO_WORK_EST_PER_CAPITA                          1.886512e-17
BIKE_TO_WORK_MARG_PER_CAPITA                        -3.773024e-17
WALK_TO_WORK_EST_PER_CAPITA                         -7.372575e-18
WALK_TO_WORK_MARG_PER_CAPITA                         9.974660e-18
DRIVE_SOLO_TO_WORK_EST_PER_CAPITA                   -7.047314e-19
DRIVE_SOLO_TO_WORK_MARG_PER_CAPITA                   5.637851e-18
CARPOOL_TO_WORK_EST_PER_CAPITA                       3.361027e-18
CARPOOL_TO_WORK_MARG_PER_CAPITA                     -1.691355e-17
PUBTRANS_TO_WORK_EST_PER_CAPITA                     -1.816039e-18
PUBTRANS_TO_WORK_MARG_PER_CAPITA                     2.688821e-17
AUTOMOBILE_COUNT_PER_CAPITA                          1.279359e-17
BICYCLE_DEATH_BY_AUTO_COUNT_PER_CAPITA               4.000000e-01
BICYCLE_SUSP_SERIOUS_INJ_BY_AUTO_COUNT_PER_CAPITA    1.000000e+00
PED_BY_AUT

### 4. Plot the residuals, display feature coefficients, and create a QQ plot

### Credit:

The following code is based on the work of my supervisor, Mitch Shiles. The original code can be found at this link: [Mitch Shiles' GitHub](https://github.com/rmshiles/Custom-Data-Tools/blob/main/testing%20for%20normality%20.ipynb).

In [98]:
# Plot the residuals, display feature coefficients, and create a QQ plot
test_for_normality(y_test,y_pred)

the Jarque berra stat is 0.2900874877560054, and the pvalue is 0.8649844545086915
NormaltestResult(statistic=0.43377594404878, pvalue=0.8050201490312022)
the Minimum is -2.757939276260002e-18
the Maximum is 1.6127507315721878e-18
the Length is 9
the Length is 9
the Standard Deviation is 1.3125990431518983e-18
the Mean is -8.664205902692321e-19




/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


**The warning regarding the kurtosis test indicates that it's typically valid for larger sample sizes (n >= 20). With a sample size of 9, it's recommended to use other statistical tests or methods to assess kurtosis accurately.**